In [1]:
from backend.stripe import stripe

In [2]:
user = User.objects.get(email='veyorokon@gmail.com')

In [3]:
customer = Customer.objects.get(subscriber=user)

In [11]:
customerapi = customer.api_retrieve()

In [12]:
customerapi

<Customer customer id=cus_Ely4ei6DSDgze2 at 0x7fa49991be60> JSON: {
  "account_balance": 0,
  "created": 1553648478,
  "currency": "usd",
  "default_source": {
    "address_city": "Got some work",
    "address_country": "DO",
    "address_line1": "Scooby dooby doo",
    "address_line1_check": "pass",
    "address_line2": "where r u",
    "address_state": "TO",
    "address_zip": "45069",
    "address_zip_check": "pass",
    "brand": "Visa",
    "country": "US",
    "customer": "cus_Ely4ei6DSDgze2",
    "cvc_check": "pass",
    "dynamic_last4": null,
    "exp_month": 11,
    "exp_year": 2022,
    "fingerprint": "n5HgGyz8F3bvMdL0",
    "funding": "credit",
    "id": "card_1EJZ4kGjkoMLHlzNaKs1Y0yS",
    "last4": "4242",
    "metadata": {
      "name_on_card": "Scooby Doo"
    },
    "name": null,
    "object": "card",
    "tokenization_method": null
  },
  "delinquent": false,
  "description": null,
  "discount": null,
  "email": "veyorokon@gmail.com",
  "id": "cus_Ely4ei6DSDgze2",
  "inv